In [34]:
#required packages
import json
import csv

In [35]:
#read and parse json files
def read_json(filename):
    with open(filename, 'r') as f:
        return [json.loads(line) for line in f]
#assigning variables for read json files
patients = read_json('Patient.ndjson')
conditions = read_json('Condition.ndjson')
encounters_normal = read_json('Encounter.ndjson')
encounters_icu = read_json('EncounterICU.ndjson')

In [36]:
#creating a dictionary to store conditions for each patient
patient_conditions = {}#condition dictionary
for patient in patients:
    patient_conditions[patient['id']] = []
# populate patient_conditions dictionary with condition information
for condition in conditions:
    patient_id = condition['subject']['reference'].split('/')[-1]  # extract patient id from reference
    patient_conditions[patient_id].append(condition)

In [37]:
#time retrieval
def get_start_time(encounter_id):
    for encounter in encounters_combined:
        if encounter['id'] == encounter_id:
            if 'period' in encounter and 'start' in encounter['period']:
                return convert_to_unix_time(encounter['period']['start'])
    return None
# function to convert datetime to Unix format
def convert_to_unix_time(datetime_str):
    dt = datetime.fromisoformat(datetime_str)
    return int(dt.timestamp())

In [38]:
for patient_id, patient_condition_list in patient_conditions.items():
    for condition in patient_condition_list:
        encounter_id = condition['encounter']['reference'].split('/')[-1]
        start_time = get_start_time(encounter_id)
        condition['estimated_time'] = start_time

In [39]:
#creating the dataset in csv
with open('data_output.csv', 'w', newline='') as csvfile:
    fieldnames = ['pid', 'time', 'code', 'description']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for patient_id, patient_condition_list in patient_conditions.items():
        for condition in patient_condition_list:
            writer.writerow({
                'pid': patient_id,
                'time': condition.get('estimated_time', ''),
                'code': condition['code']['coding'][0]['code'],
                'description': condition['code']['coding'][0]['display']
            })
print("the required csv data file has been generated")

the required csv data file has been generated
